<a href="https://colab.research.google.com/github/SaiRajesh228/DeepLearningAssignment1/blob/main/q11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
from keras.datasets import fashion_mnist
import wandb

In [2]:
!wandb login

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


CommError: Run initialization has timed out after 90.0 sec. Please try increasing the timeout with the `init_timeout` setting: `wandb.init(settings=wandb.Settings(init_timeout=120))`.